## PART 1:

In [2]:
import numpy as np
import pandas as pd
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

#### Loading data from Wiki

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = urlopen(url).read().decode('utf-8')
soup = BeautifulSoup(page, 'html.parser')
wiki_table = soup.body.table.tbody
wiki_table

<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North_York" tit

In [4]:
def get_cell(element):
    cells = element.find_all('td')
    row = []
    
    for cell in cells:
        if cell.a:            
            if (cell.a.text):
                row.append(cell.a.text)
                continue
        row.append(cell.string.strip())
        
    return row
def get_row():    
    data = []  
    
    for tr in wiki_table.find_all('tr'):
        row = get_cell(tr)
        if len(row) != 3:
            continue
        data.append(row)        
    
    return data

In [94]:
data = get_row()
columns = ['Postcode', 'Borough', 'Neighbourhood']
df = pd.DataFrame(data, columns=columns)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [95]:
df1 = df[df.Borough != 'Not assigned']
df1.reset_index(inplace=True)
df1.drop('index',axis=1,inplace=True)
df1.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough  
Find a cell which has a not assigned neighborhood and change the neighborhood from a not assigned to the name in the borough

In [96]:
df1[df1["Neighbourhood"]=="Not assigned"]

,Postcode,Borough,Neighbourhood
6,M7A,Queen's Park,Not assigned


In [97]:
df1.iloc[6,2]="Queen's Park"

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [98]:
df1.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


### Cleaning data

In [99]:
i = 1
while(i < len(df1)):
    if df1['Postcode'].iloc[i] == df1['Postcode'].iloc[i - 1]:
        df1.at[i - 1, 'Neighbourhood'] = df1.Neighbourhood.iloc[i - 1] +', ' + df1.Neighbourhood.iloc[i]
        df1.drop(index = i, inplace = True)
        df1 = df1.reset_index().drop('index', axis = 1)
    else:
        i += 1

In [100]:
df1.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [101]:
df1.shape

(103, 3)

## PART 2
Latitude and Longitude of each PostalCode

In [105]:
Geo=pd.read_csv("http://cocl.us/Geospatial_data")

In [106]:
Geo.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [109]:
Geo.rename({'Postal Code': 'Postcode'}, axis='columns', inplace=True)
df3 = pd.merge(df1, Geo, on='Postcode')

In [110]:
df3.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


###  PART 3  
Create a map of Toronto with neighborhoods superimposed on top

In [115]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          90 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.49   | 32 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

# All requested packages

In [116]:
CLIENT_ID = '1OYRVFGT1SGDQKEICCQGC2ICNWQFJTZFXQCIMGZFAGCDYIV3' # your Foursquare ID
CLIENT_SECRET = 'Q5T0V1JXSQB5PUBNB5ZE44DZ4PLSNYCPT5HGWMMMUS4ZI22J' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1OYRVFGT1SGDQKEICCQGC2ICNWQFJTZFXQCIMGZFAGCDYIV3
CLIENT_SECRET:Q5T0V1JXSQB5PUBNB5ZE44DZ4PLSNYCPT5HGWMMMUS4ZI22J


Let's explore the first neighborhood in our dataframe.  
Get the neighborhood's name.

In [119]:
df3.loc[0, 'Neighbourhood']

'Parkwoods'

Get the neighborhood's latitude and longitude values.

In [124]:
neighbourhood_latitude = df3.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = df3.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = df3.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.  
First, let's create the GET request URL. Name your URL url

In [125]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url 

'https://api.foursquare.com/v2/venues/explore?&client_id=1OYRVFGT1SGDQKEICCQGC2ICNWQFJTZFXQCIMGZFAGCDYIV3&client_secret=Q5T0V1JXSQB5PUBNB5ZE44DZ4PLSNYCPT5HGWMMMUS4ZI22J&v=20180605&ll=43.7532586,-79.3296565&radius=500&limit=100'

In [126]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d3b610ca197930038c994bd'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': 

In [127]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [128]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,KFC,Fast Food Restaurant,43.754387,-79.333021
2,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [129]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


#### Explore Neighborhoods in Toronto

In [131]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    print("Finished!")
    
    return(nearby_venues)

In [134]:
# type your answer here

Toronto_venues = getNearbyVenues(names=df3['Neighbourhood'],
                                 latitudes=df3['Latitude'],
                                 longitudes=df3['Longitude'],
                                 radius = 2000
                                )

Finished!


In [135]:
print(Toronto_venues.shape)
Toronto_venues.head()

(8565, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.753259,-79.329656,Donalda Golf & Country Club,43.752816,-79.342741,Golf Course
2,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
3,Parkwoods,43.753259,-79.329656,Island Foods,43.745866,-79.346035,Caribbean Restaurant
4,Parkwoods,43.753259,-79.329656,Graydon Hall Manor,43.763923,-79.342961,Event Space


In [136]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,100,100,100,100,100,100
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",98,98,98,98,98,98
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",46,46,46,46,46,46
"Alderwood, Long Branch",100,100,100,100,100,100
"Bathurst Manor, Downsview North, Wilson Heights",56,56,56,56,56,56
Bayview Village,50,50,50,50,50,50
"Bedford Park, Lawrence Manor East",100,100,100,100,100,100
Berczy Park,100,100,100,100,100,100


In [137]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 323 uniques categories.


In [138]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Zoo Exhibit,Accessories Store,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Cemetery,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,College Stadium,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Post Office,Poutine Place,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,River,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports 

In [139]:
Toronto_onehot.shape

(8565, 323)

In [140]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped.head()

,Neighborhood,Zoo Exhibit,Accessories Store,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Cemetery,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,College Stadium,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Post Office,Poutine Place,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,River,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports 

In [145]:
Toronto_grouped.shape

(103, 323)

In [141]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
         venue  freq
0  Coffee Shop  0.07
1        Hotel  0.05
2    Gastropub  0.04
3         Café  0.04
4      Theater  0.04


----Agincourt----
                venue  freq
0  Chinese Restaurant  0.12
1         Coffee Shop  0.08
2            Pharmacy  0.05
3          Restaurant  0.04
4              Bakery  0.03


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                   venue  freq
0     Chinese Restaurant  0.19
1            Coffee Shop  0.07
2                   Park  0.05
3                 Bakery  0.04
4  Vietnamese Restaurant  0.04


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                  venue  freq
0           Coffee Shop  0.17
1  Fast Food Restaurant  0.11
2           Pizza Place  0.11
3     Indian Restaurant  0.09
4         Grocery Store  0.07


----Alderwood, Long Branch----
                  venue  freq
0           Coffee Shop  0.08
1      

In [152]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [153]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Hotel,Café,Gastropub,Theater,Restaurant,Beer Bar,Pizza Place,Vegetarian / Vegan Restaurant,Burrito Place
1,Agincourt,Chinese Restaurant,Coffee Shop,Pharmacy,Restaurant,Sandwich Place,Shopping Mall,Bakery,Indian Restaurant,Ice Cream Shop,Fast Food Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Chinese Restaurant,Coffee Shop,Park,Vietnamese Restaurant,Pizza Place,Bakery,Japanese Restaurant,Bubble Tea Shop,Korean Restaurant,Pharmacy
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Coffee Shop,Fast Food Restaurant,Pizza Place,Indian Restaurant,Grocery Store,Park,Skating Rink,Restaurant,Sandwich Place,Steakhouse
4,"Alderwood, Long Branch",Coffee Shop,Pizza Place,Department Store,Fast Food Restaurant,Pharmacy,Clothing Store,Breakfast Spot,Burger Joint,Seafood Restaurant,Café


In [143]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:]

array([1, 2, 2, 0, 2, 2, 2, 2, 1, 2, 2, 1, 1, 2, 1, 1, 2, 2, 2, 1, 1, 1,
       1, 0, 0, 0, 2, 1, 1, 1, 1, 2, 1, 2, 0, 1, 0, 0, 0, 0, 0, 1, 0, 2,
       1, 2, 1, 2, 0, 1, 1, 1, 1, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2,
       1, 2, 1, 2, 1, 2, 2, 1, 2, 1, 1, 1, 4, 1, 1, 0, 2, 1, 1, 1, 1, 2,
       1, 1, 1, 2, 2, 3, 0, 0, 0, 1, 2, 0, 0, 2, 2], dtype=int32)

In [150]:
Toronto_merged = df3

# add clustering labels
Toronto_merged['Cluster Labels'] = kmeans.labels_

In [151]:
Toronto_merged.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels
0,M3A,North York,Parkwoods,43.753259,-79.329656,1
1,M4A,North York,Victoria Village,43.725882,-79.315572,2
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,2
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,0
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,2
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242,2
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,2
7,M3B,North York,Don Mills North,43.745906,-79.352188,2
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937,1
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,2


In [156]:
Toronto_merged.columns=['Postcode','Borough','Neighborhood','Latitude','Longitude','Cluster Labels']
Toronto_merged.head(10)

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels
0,M3A,North York,Parkwoods,43.753259,-79.329656,1
1,M4A,North York,Victoria Village,43.725882,-79.315572,2
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,2
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,0
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,2
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242,2
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,2
7,M3B,North York,Don Mills North,43.745906,-79.352188,2
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937,1
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,2


In [158]:

# add clustering labels
Toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1,Coffee Shop,Japanese Restaurant,Pizza Place,Pharmacy,Discount Store,Sandwich Place,Chinese Restaurant,Supermarket,Fried Chicken Joint,Intersection
1,M4A,North York,Victoria Village,43.725882,-79.315572,2,Coffee Shop,Fast Food Restaurant,Gym,Clothing Store,Grocery Store,Sandwich Place,Discount Store,Japanese Restaurant,Middle Eastern Restaurant,Bookstore
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,2,Coffee Shop,Café,Park,Restaurant,Japanese Restaurant,Bakery,Farmers Market,Italian Restaurant,Thai Restaurant,Breakfast Spot
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,0,Clothing Store,Coffee Shop,Fast Food Restaurant,Grocery Store,Vietnamese Restaurant,Furniture / Home Store,Restaurant,Dessert Shop,Pet Store,Liquor Store
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,2,Café,Coffee Shop,Vegetarian / Vegan Restaurant,Pizza Place,Japanese Restaurant,Mexican Restaurant,Park,Gastropub,Breakfast Spot,Theater


In [159]:
# create map
map_clusters = folium.Map(location=[43.706204, -79.398752], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [160]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,North York,0,Clothing Store,Coffee Shop,Fast Food Restaurant,Grocery Store,Vietnamese Restaurant,Furniture / Home Store,Restaurant,Dessert Shop,Pet Store,Liquor Store
23,East York,0,Coffee Shop,Indian Restaurant,Bakery,Grocery Store,Restaurant,Burger Joint,Supermarket,Turkish Restaurant,Sporting Goods Shop,Thai Restaurant
24,Downtown Toronto,0,Coffee Shop,Café,Pizza Place,Theater,Restaurant,Vegetarian / Vegan Restaurant,Park,Concert Hall,Breakfast Spot,Arts & Crafts Store
25,Downtown Toronto,0,Café,Coffee Shop,Bar,Korean Restaurant,Grocery Store,Vegetarian / Vegan Restaurant,Indian Restaurant,Cocktail Bar,Restaurant,Beer Bar
34,North York,0,Coffee Shop,Restaurant,Sandwich Place,Grocery Store,Pharmacy,Furniture / Home Store,Bank,Chinese Restaurant,Japanese Restaurant,Italian Restaurant
36,Downtown Toronto,0,Café,Coffee Shop,Hotel,Park,Vegetarian / Vegan Restaurant,Thai Restaurant,Baseball Stadium,Gastropub,Seafood Restaurant,Japanese Restaurant
37,West Toronto,0,Café,Bar,Cocktail Bar,Restaurant,Coffee Shop,Bakery,Pizza Place,Italian Restaurant,Asian Restaurant,Furniture / Home Store
38,Scarborough,0,Coffee Shop,Fast Food Restaurant,Grocery Store,Chinese Restaurant,Pharmacy,Discount Store,Beer Store,Bank,Sandwich Place,Gas Station
39,North York,0,Chinese Restaurant,Coffee Shop,Bank,Park,Pharmacy,Café,Trail,Shopping Mall,Grocery Store,Sandwich Place
40,North York,0,Clothing Store,Coffee Shop,Athletics & Sports,American Restaurant,Cosmetics Shop,Vietnamese Restaurant,Gym / Fitness Center,Furniture / Home Store,Flea Market,Pharmacy


In [161]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1,Coffee Shop,Japanese Restaurant,Pizza Place,Pharmacy,Discount Store,Sandwich Place,Chinese Restaurant,Supermarket,Fried Chicken Joint,Intersection
8,East York,1,Pizza Place,Park,Fast Food Restaurant,Coffee Shop,Sandwich Place,Brewery,Pharmacy,Beer Store,Skating Rink,Grocery Store
11,Etobicoke,1,Coffee Shop,Convenience Store,Grocery Store,Park,Bank,Pizza Place,Fish & Chips Shop,Sandwich Place,Sushi Restaurant,Pharmacy
12,Scarborough,1,Breakfast Spot,Coffee Shop,Sandwich Place,Pizza Place,Pharmacy,Pet Store,Ice Cream Shop,Bank,Beer Store,Supermarket
14,East York,1,Pizza Place,Coffee Shop,Park,Café,Gastropub,Ice Cream Shop,Thai Restaurant,Bakery,Pharmacy,Ethiopian Restaurant
15,Downtown Toronto,1,Coffee Shop,Café,Hotel,Gastropub,Farmers Market,Japanese Restaurant,Italian Restaurant,Vegetarian / Vegan Restaurant,Thai Restaurant,Pizza Place
19,East Toronto,1,Coffee Shop,Pub,Beach,Breakfast Spot,Bar,Bakery,BBQ Joint,Japanese Restaurant,Park,Café
20,Downtown Toronto,1,Coffee Shop,Café,Hotel,Japanese Restaurant,Park,Gastropub,Italian Restaurant,Farmers Market,Seafood Restaurant,Bistro
21,York,1,Italian Restaurant,Sandwich Place,Bakery,Pizza Place,Coffee Shop,Furniture / Home Store,Bank,Mexican Restaurant,Breakfast Spot,Grocery Store
22,Scarborough,1,Coffee Shop,Fast Food Restaurant,Sandwich Place,Pizza Place,Beer Store,Park,Pharmacy,Bakery,Discount Store,Chinese Restaurant


In [162]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,2,Coffee Shop,Fast Food Restaurant,Gym,Clothing Store,Grocery Store,Sandwich Place,Discount Store,Japanese Restaurant,Middle Eastern Restaurant,Bookstore
2,Downtown Toronto,2,Coffee Shop,Café,Park,Restaurant,Japanese Restaurant,Bakery,Farmers Market,Italian Restaurant,Thai Restaurant,Breakfast Spot
4,Queen's Park,2,Café,Coffee Shop,Vegetarian / Vegan Restaurant,Pizza Place,Japanese Restaurant,Mexican Restaurant,Park,Gastropub,Breakfast Spot,Theater
5,Etobicoke,2,Grocery Store,Park,Pharmacy,Coffee Shop,Café,Bank,Liquor Store,Shopping Mall,Golf Course,Pet Store
6,Scarborough,2,Zoo Exhibit,Fast Food Restaurant,Pizza Place,Athletics & Sports,Other Great Outdoors,Zoo,Spa,Park,Skating Rink,Bus Station
7,North York,2,Coffee Shop,Japanese Restaurant,Park,Pizza Place,Restaurant,Bank,Italian Restaurant,Burger Joint,Supermarket,Bar
9,Downtown Toronto,2,Coffee Shop,Café,Gastropub,Vegetarian / Vegan Restaurant,Japanese Restaurant,Pizza Place,Restaurant,Steakhouse,Italian Restaurant,Thai Restaurant
10,North York,2,Clothing Store,Coffee Shop,Furniture / Home Store,Fast Food Restaurant,Grocery Store,Fried Chicken Joint,Japanese Restaurant,Bakery,Bank,Chocolate Shop
13,North York,2,Coffee Shop,Park,Japanese Restaurant,Sandwich Place,Middle Eastern Restaurant,Restaurant,Gym,Beer Store,Italian Restaurant,Movie Theater
16,York,2,Coffee Shop,Italian Restaurant,Bank,Café,Mexican Restaurant,Bakery,Ice Cream Shop,Indian Restaurant,Caribbean Restaurant,Bagel Shop


In [163]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
93,Etobicoke,3,Coffee Shop,Pizza Place,Department Store,Fast Food Restaurant,Pharmacy,Clothing Store,Breakfast Spot,Burger Joint,Seafood Restaurant,Café


In [164]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
78,Scarborough,4,Chinese Restaurant,Coffee Shop,Pharmacy,Restaurant,Sandwich Place,Shopping Mall,Bakery,Indian Restaurant,Ice Cream Shop,Fast Food Restaurant
